# Setup

In [3]:
import pandas as pd
import numpy as np
import os 
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
import random
import time

In [4]:
data_file = "output2.txt"
output_dir = Path(f"./output_dir/")

In [5]:
import re
def parse_output_file(filepath):
    text =''
    paths,contents = [],[]
    with open(filepath, 'r',encoding='utf-8') as f:
        text = f.read()
    matches = re.finditer(r'C:/.*',text)
    matches1 = [match for match in matches]# make this the last section of the codebase
    for match in matches1:
        match = match.group(0)
        match = match.split('/')[-1]
        paths.append(match)
    for match, match_next in zip(matches1[:-1],matches1[1:]):
        ending_index = match.span()[1]
        starting_index = match_next.span()[0]
        contents.append(text[ending_index:starting_index])
    return paths,contents
    #appending the last file
    # unique_dirs = set()
    # contents.append(text[matches1[-1].span()[1]:])
    # for path in paths:
    #     path = path.replace('/','\\')
    #     dir = path.split('\\')  
    #     dir = dir[-heirarchy]
    #     unique_dirs.add(dir)
    
    # dir_contents = {}
    # for dir in unique_dirs:
    #     dir_contents[dir] = []
    # file_names = {}
    # for path, content in zip(paths, contents):
    #     path = path.replace('/', '\\')
    #     dir = path.split('\\')[-heirarchy]
    #     dir_contents[dir]+=str(content)
    #     file_names[dir] = path.split('\\')[-1]
    
    # return file_names, dir_contents, unique_dirs
paths,contents = parse_output_file(data_file)
print(paths[0],contents[0][:10])

vram\CryptoPkg\Test\UnitTest\Library\BaseCryptLib\TSTests.c 
/** @file


In [6]:
from langchain.schema import Document
class CustomTextLoader(TextLoader):
    def __init__(self, file_path, **kwargs):
        super().__init__(file_path, **kwargs)
        self.file_path = file_path

    def load(self):
        paths, contents = parse_output_file(self.file_path)
        documents = []
        for path, content in zip(paths, contents):
            metadata = {"source": path}
            documents.append(Document(page_content=content.strip(), metadata=metadata))
        return documents
loader = CustomTextLoader(data_file, encoding='utf-8')
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=5000,
    chunk_overlap = 100,
    length_function = len,
    is_separator_regex=False
)
pages = splitter.split_documents(documents)
print(len(pages))
pages = pages[10000:10100]  
print(pages[3].page_content[:10])

13759
/* resetin


In [7]:
import httpx
api_key = os.getenv('API_KEY')
base_url = os.getenv('API_URL')
max_output_tokens = 1000
streaming = False
http_client = httpx.Client(verify=False)
available_models = [
    "mixtral-8x7b-instruct-v01", 
    "gemma-7b-it", 
    "mistral-7b-instruct-v02", 
    "llama-2-70b-chat", 
    "phi-3-mini-128k-instruct", 
    "llama-3-8b-instruct"]

In [8]:
import sys
import json
from yachalk import chalk
from langchain_openai import ChatOpenAI,OpenAI
llm = OpenAI(
    base_url=base_url,
    model=available_models[0],
    http_client=http_client,
    api_key=api_key,
    max_tokens = 1000,
)

# Creating all the utility functions

In [12]:


# Append the parent directory to the system path
sys.path.append("..")

# Initialize the ChatOpenAI client

def trim_incomplete_json(json_string):
    # Find the last occurrence of '}]' or '},' in the string
    last_complete = max(json_string.rfind('}]'), json_string.rfind('},'))
    
    if last_complete != -1:
        # If found, trim the string to that point and add closing bracket if needed
        trimmed = json_string[:last_complete+1]
        if not trimmed.endswith(']'):
            trimmed += ']'
        return trimmed
    else:
        # If no complete object found, return empty list
        return '[]'

def extract_concepts(prompt: str, metadata: dict = {}) -> list:
    SYS_PROMPT = (
        "Your task is to extract the key concepts (and non-personal entities) mentioned in the given context. "
        "Extract only the most important and atomistic concepts, breaking them down into simpler concepts if needed. "
        "Categorize the concepts into one of the following categories: "
        "[import statement, concept, function definition, object-calling, document, class-definition, condition, misc].\n"
        "Format your output as a list of JSON objects in the following format:\n"
        "[\n"
        "   {\n"
        '       "entity": "The Concept",\n'
        '       "importance": "The contextual importance of the concept on a scale of 1 to 5 (5 being the highest)",\n'
        '       "category": "The Type of Concept"\n'
        "   },\n"
        "   {...}\n"
        "]"
    )

    messages = [
        {"role": "system", "content": SYS_PROMPT},
        {"role": "user", "content": prompt}
    ]
    
    response = llm.invoke(input=messages)
    print("Extract Prompt ", response)
    try:
        result = json.loads(response)
    except json.JSONDecodeError:
        print("\n\nWARNING ### Incomplete JSON detected. Attempting to trim...")
        trimmed_response = trim_incomplete_json(response)
        print(trimmed_response+"\n#####################################################################################################")
        try:
            result = json.loads(trimmed_response)
        except json.JSONDecodeError:
            print("\n\nERROR ### Failed to parse even after trimming. Here is the buggy response: ", response, "\n\n")
            return None

    if result is not None:
        result = [dict(item, **metadata) for item in result]

    return result

def graph_prompt(input_text: str, metadata: dict = {}) -> list:
    SYS_PROMPT = (
        "You are a network graph maker who extracts terms and their relations from a given context. "
        "You are provided with a context chunk (delimited by ```). Your task is to extract the ontology "
        "of terms mentioned in the given context. These terms should represent the key concepts according to the context.\n"
        "Thought 1: While traversing through each sentence, think about whether Data is being passed to it\n"
        "\tTerms may include object creation, entity, class definition, import file, function signature, \n"
        "\tcondition, parameters, documents, service, concept, etc.\n"
        "\tTerms should be as concise as possible but ignore vague definitions\n\n"
        "Thought 2: Think about how these terms can have one-on-one relations with other terms.\n"
        "\tTerms mentioned in the same code or file are typically related to each other.\n"
        "\tTerms can be related to many other terms.\n\n"
        "Thought 3: Determine the relation between each related pair of terms.\n\n"
        "Format your output as a list of JSON objects. Each element of the list contains a pair of terms do not provide an explanation, JUST THE JSON OUTPUT "
        "and the relationship between them, as follows:\n"
        "[\n"
        "   {\n"
        '       "node_1": "A concept from the extracted ontology",\n'
        '       "node_2": "A related concept from the extracted ontology",\n'
        '       "edge": "The relationship between node_1 and node_2 in one or two sentences"\n'
        "   },\n"
        "   {...}\n"
        "]"
    )

    USER_PROMPT = f"context: ```{input_text}``` \n\n output: "

    messages = [
        {"role": "system", "content": SYS_PROMPT},
        {"role": "user", "content": USER_PROMPT}
    ]

    response = llm.invoke(input=messages)
    # print("Graph Prompt ", response)
    try:
        result = json.loads(response)
    except json.JSONDecodeError:
        # print("\n\nWARNING ### Incomplete JSON detected. Attempting to trim...")
        trimmed_response = trim_incomplete_json(response)
        # print(trimmed_response)
        # print("################################################################################################################")
        try:
            result = json.loads(trimmed_response)
        except json.JSONDecodeError:
            print("\n\nERROR ### Failed to parse even after trimming. Here is the buggy response: ")
            return None

    if result is not None:
        result = [dict(item, **metadata) for item in result]

    return result

## Dataframe and graph manipulation

In [50]:
import uuid
import pandas as pd
import numpy as np


def documents2Dataframe(documents) -> pd.DataFrame:
    rows = []
    for chunk in documents:
        row = {
            "text": chunk.page_content,
            "path": chunk.metadata["source"],
            **chunk.metadata,
            "chunk_id": uuid.uuid4().hex,
        }
        rows = rows + [row]

    df = pd.DataFrame(rows)
    return df


def df2ConceptsList(dataframe: pd.DataFrame) -> list:
    # dataframe.reset_index(inplace=True)
    results = dataframe.apply(
        lambda row: extract_concepts(
            row.text, {"chunk_id": row.chunk_id, "type": "concept"}
        ),
        axis=1,
    )
    # invalid json results in NaN
    results = results.dropna()
    results = results.reset_index(drop=True)

    ## Flatten the list of lists to one single list of entities.
    concept_list = np.concatenate(results).ravel().tolist()
    return concept_list


def concepts2Df(concepts_list) -> pd.DataFrame:
    ## Remove all NaN entities
    concepts_dataframe = pd.DataFrame(concepts_list).replace(" ", np.nan)
    concepts_dataframe = concepts_dataframe.dropna(subset=["entity"])
    concepts_dataframe["entity"] = concepts_dataframe["entity"].apply(
        lambda x: x.lower()
    )

    return concepts_dataframe


def df2Graph(dataframe: pd.DataFrame, model=None) -> list:
    total_rows = len(dataframe)
    processed_rows = 0
    start_time = time.time()

    def process_row(row):
        nonlocal processed_rows
        result = graph_prompt(row.text, {"chunk_id": row.chunk_id,"path":row.path})
        processed_rows += 1
        elapsed_time = time.time() - start_time
        avg_time_per_row = elapsed_time / processed_rows
        estimated_time_remaining = (total_rows - processed_rows) * avg_time_per_row

        print(f"\rProcessing: {processed_rows}/{total_rows} rows | "
              f"Elapsed: {elapsed_time:.2f}s | "
              f"Estimated time remaining: {estimated_time_remaining:.2f}s", 
              end="", flush=True)
        return result

    results = dataframe.apply(process_row, axis=1)

    print("\nProcessing complete!")
    # print(results)

    # Filter out None values and flatten the list of lists to one single list of entities.
    concept_list = [item for sublist in results if sublist is not None for item in sublist]
    return concept_list

def graph2Df(nodes_list) -> pd.DataFrame:
    ## Remove all NaN entities
    graph_dataframe = pd.DataFrame(nodes_list).replace(" ", np.nan)
    graph_dataframe = graph_dataframe.dropna(subset=["node_1", "node_2"])
    graph_dataframe["node_1"] = graph_dataframe["node_1"].apply(lambda x: x.lower())
    graph_dataframe["node_2"] = graph_dataframe["node_2"].apply(lambda x: x.lower())

    return graph_dataframe

In [51]:

df = documents2Dataframe(pages)

100


# Actual processing

In [72]:
## To regenerate the graph with LLM, set this to True
regenerate = True

if regenerate:
    concepts_list = df2Graph(df)
    dfg1 = graph2Df(concepts_list)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    dfg1.to_csv(output_dir/"graph.csv", sep="|", index=False)
    df.to_csv(output_dir/"chunks.csv", sep="|", index=False)
else:
    dfg1 = pd.read_csv(output_dir/"graph.csv", sep="|")

dfg1.replace("", np.nan, inplace=True)
dfg1.dropna(subset=["node_1", "node_2", 'edge'], inplace=True)
dfg1['count'] = 4 
## Increasing the weight of the relation to 4. 
## We will assign the weight of 1 when later the contextual proximity will be calculated.  


Processing: 16/100 rows | Elapsed: 189.94s | Estimated time remaining: 997.19ss

ERROR ### Failed to parse even after trimming. Here is the buggy response: 
Processing: 33/100 rows | Elapsed: 389.74s | Estimated time remaining: 791.30s

ERROR ### Failed to parse even after trimming. Here is the buggy response: 
Processing: 68/100 rows | Elapsed: 839.90s | Estimated time remaining: 395.25s

ERROR ### Failed to parse even after trimming. Here is the buggy response: 
Processing: 100/100 rows | Elapsed: 1212.09s | Estimated time remaining: 0.00ss
Processing complete!
0     [{'node_1': 'libspdm_test_requester_encap_cert...
1     [{'node_1': 'spdm_test_context', 'node_2': 'sp...
2     [{'node_1': 'm_spdm_get_certificate_request3.l...
3     [{'node_1': 'spdm_context', 'node_2': 'libspdm...
4     [{'node_1': 'libspdm_test_context_t', 'node_2'...
                            ...                        
95    [{'node_1': 'libspdm_requester_encap_get_diges...
96    [{'node_1': 'libspdm_test_contex

## connecting node with more contextual proximity

adding count to the edges to design strength

In [74]:
def contextual_proximity(df: pd.DataFrame) -> pd.DataFrame:
    # Melt the dataframe into a list of nodes
    dfg_long = pd.melt(
        df, id_vars=["chunk_id", "path"], value_vars=["node_1", "node_2"], value_name="node"
    )
    dfg_long.drop(columns=["variable"], inplace=True)

    # Self join with chunk id as the key will create a link between terms occurring in the same text chunk.
    dfg_wide = pd.merge(dfg_long, dfg_long, on=["chunk_id", "path"], suffixes=("_1", "_2"))

    # drop self loops
    self_loops_drop = dfg_wide[dfg_wide["node_1"] == dfg_wide["node_2"]].index
    dfg2 = dfg_wide.drop(index=self_loops_drop).reset_index(drop=True)

    # Group and count edges.
    dfg2 = (
        dfg2.groupby(["node_1", "node_2"])
        .agg({"chunk_id": [",".join, "count"], "path": "first"})
        .reset_index()
    )
    dfg2.columns = ["node_1", "node_2", "chunk_id", "count", "path"]
    dfg2.replace("", np.nan, inplace=True)
    dfg2.dropna(subset=["node_1", "node_2"], inplace=True)
    # Drop edges with 1 count
    dfg2 = dfg2[dfg2["count"] != 1]
    dfg2["edge"] = "contextual proximity"
    return dfg2


dfg2 = contextual_proximity(dfg1)

                           chunk_id  \
0  bce693c03f8f47189cc4debbabb4aeb0   
1  bce693c03f8f47189cc4debbabb4aeb0   
2  dea3c0da87b64f3e90edcdd84b93e12e   
3  dea3c0da87b64f3e90edcdd84b93e12e   
4  dea3c0da87b64f3e90edcdd84b93e12e   

                                                path variable  \
0  vram\SecurityPkg\DeviceSecurity\SpdmLib\libspd...   node_1   
1  vram\SecurityPkg\DeviceSecurity\SpdmLib\libspd...   node_1   
2  vram\SecurityPkg\DeviceSecurity\SpdmLib\libspd...   node_1   
3  vram\SecurityPkg\DeviceSecurity\SpdmLib\libspd...   node_1   
4  vram\SecurityPkg\DeviceSecurity\SpdmLib\libspd...   node_1   

                                             node  
0  libspdm_test_requester_encap_certificate_case1  
1  libspdm_test_requester_encap_certificate_case1  
2                               spdm_test_context  
3                                    spdm_context  
4                   spdm_context->connection_info  


,node_1,node_2,edge,chunk_id,path,count
327,connection_info,version,connection_info contains version,7e313f371d84458091e17ab98b821fc8,vram\SecurityPkg\DeviceSecurity\SpdmLib\libspd...,4
328,connection_info,connection_state,connection_info contains connection_state,7e313f371d84458091e17ab98b821fc8,vram\SecurityPkg\DeviceSecurity\SpdmLib\libspd...,4
329,libspdm_requester_encap_digests_test_main,int,libspdm_requester_encap_digests_test_main is a...,892d8804db6a45d8980ba2a37aff41ae,vram\SecurityPkg\DeviceSecurity\SpdmLib\libspd...,4
330,libspdm_requester_encap_digests_test_main,void,libspdm_requester_encap_digests_test_main is a...,892d8804db6a45d8980ba2a37aff41ae,vram\SecurityPkg\DeviceSecurity\SpdmLib\libspd...,4
331,cmocka_unit_test,test_spdm_requester_encap_get_digests_case1,cmocka_unit_test is a macro that takes test_sp...,892d8804db6a45d8980ba2a37aff41ae,vram\SecurityPkg\DeviceSecurity\SpdmLib\libspd...,4


In [75]:
dfg = pd.concat([dfg1, dfg2], axis=0)
dfg = (
    dfg.groupby(["node_1", "node_2"])
    .agg({
        "chunk_id": ",".join,
        "edge": ",".join,
        "count": "sum",
        "path": lambda x: ",".join(set(x))  # Combine unique paths
    })
    .reset_index()
)


,node_1,node_2,chunk_id,edge,count,path
0,((libspdm_context_t*)spdm_context)->local_cont...,libspdm_read_responder_public_certificate_chain,"41c0a30e7e5a4bd1ae7bd8cc63a2d818,41c0a30e7e5a4...",contextual proximity,5,vram\SecurityPkg\DeviceSecurity\SpdmLib\libspd...
1,**state,libspdm_return_t,"d703f95925e341129f145f6f3d026d76,d703f95925e34...",contextual proximity,2,vram\SecurityPkg\DeviceSecurity\SpdmLib\libspd...
2,**state,libspdm_test_requester_encap_request_case4,"d703f95925e341129f145f6f3d026d76,d703f95925e34...",contextual proximity,2,vram\SecurityPkg\DeviceSecurity\SpdmLib\libspd...
3,**state,void,"d703f95925e341129f145f6f3d026d76,d703f95925e34...",contextual proximity,2,vram\SecurityPkg\DeviceSecurity\SpdmLib\libspd...
4,*state,libspdm_return_t,"a3bac0cacfa9420c8271599e6d231d50,d703f95925e34...",contextual proximity,3,vram\SecurityPkg\DeviceSecurity\SpdmLib\libspd...
...,...,...,...,...,...,...
704,void,libspdm_test_context_t,"d703f95925e341129f145f6f3d026d76,33cb62ae19e44...",contextual proximity,6,vram\SecurityPkg\DeviceSecurity\SpdmLib\libspd...
705,void,libspdm_test_requester_encap_request_case4,"d703f95925e341129f145f6f3d026d76,d703f95925e34...",contextual proximity,4,vram\SecurityPkg\DeviceSecurity\SpdmLib\libspd...
706,void,spdm_dispatch,"fa142ed632d7457bb252e6135c8dfcf0,fa142ed632d74...",contextual proximity,3,vram\SecurityPkg\DeviceSecurity\SpdmLib\libspd...
707,void,spdm_server_init,"fa142ed632d7457bb252e6135c8dfcf0,fa142ed632d74...",contextual proximity,2,vram\SecurityPkg\DeviceSecurity\SpdmLib\libspd...


In [77]:
nodes = pd.concat([dfg['node_1'], dfg['node_2']], axis=0).unique()
nodes.shape

(181,)

# Drawing the graph

In [78]:
import networkx as nx
G = nx.Graph()

## Add nodes to the graph
for node in nodes:
    G.add_node(
        str(node)
    )

## Add edges to the graph
for index, row in dfg.iterrows():
    G.add_edge(
        str(row["node_1"]),
        str(row["node_2"]),
        title=row["edge"],
        weight=row['count']/4
    )

In [79]:
communities_generator = nx.community.girvan_newman(G)
top_level_communities = next(communities_generator)
next_level_communities = next(communities_generator)
communities = sorted(map(sorted, next_level_communities))
print("Number of Communities = ", len(communities))
print(communities)

Number of Communities =  11
[['((libspdm_context_t*)spdm_context)->local_context.local_cert_chain_provision_size[0]', '**state', '*state', '0', 'algorithm', 'base_asym_algo', 'base_hash_algo', 'buffer', 'buffer_size', 'capability', 'case_id', 'connection_info', 'connection_state', 'data', 'data_size', 'error_code', 'false', 'file_data', 'file_name', 'flags', 'libspdm_challenge', 'libspdm_context', 'libspdm_context_t', 'libspdm_context_t*', 'libspdm_context_t* spdm_context', 'libspdm_dump_hex_str', 'libspdm_get_asym_signature_size', 'libspdm_get_encap_response_certificate', 'libspdm_get_encap_response_key_update', 'libspdm_get_hash_size', 'libspdm_get_measurement', 'libspdm_hash_all', 'libspdm_read_input_file', 'libspdm_read_responder_public_certificate_chain', 'libspdm_read_responder_public_certificate_chain_by_size', 'libspdm_requester_challenge_test_receive_message', 'libspdm_requester_challenge_test_send_message', 'libspdm_requester_encap_get_digests_case1', 'libspdm_requester_encap

In [80]:
import seaborn as sns
palette = "hls"

## Now add these colors to communities and make another dataframe
def colors2Community(communities) -> pd.DataFrame:
    ## Define a color palette
    p = sns.color_palette(palette, len(communities)).as_hex()
    random.shuffle(p)
    rows = []
    group = 0
    for community in communities:
        color = p.pop()
        group += 1
        for node in community:
            rows += [{"node": node, "color": color, "group": group}]
    df_colors = pd.DataFrame(rows)
    return df_colors


colors = colors2Community(communities)


In [81]:
for index, row in colors.iterrows():
    G.nodes[row['node']]['group'] = row['group']
    G.nodes[row['node']]['color'] = row['color']
    G.nodes[row['node']]['size'] = G.degree[row['node']]

In [82]:
import networkx as nx
from pyvis.network import Network


k_value = 2  # Adjust this value to change node spacing
pos = nx.spring_layout(G, k=k_value, iterations=50)


net = Network(notebook=False, cdn_resources="remote", height="1200px", width="1200px")


for node, (x, y) in pos.items():
    net.add_node(
        node,
        x=x * 200, 
        y=y * 200,
        physics=True,  
        **G.nodes[node] 
    )

# Add edges to the Pyvis network
for source, target, edge_attrs in G.edges(data=True):
    edge_data = edge_attrs.copy()
    edge_data['value'] = edge_data['weight']
    net.add_edge(
        source,
        target,
        **edge_data
    )

# Disable physics in Pyvis to maintain the NetworkX layout
net.toggle_physics(True)

# Save the network
net.show("./docs/index.html", notebook=False)

./docs/index.html


# Making a Final Dataframe

In [86]:
community_map = colors.set_index('node')['group'].to_dict()


# Create new columns with default values
dfg['community_node_1'] = dfg['node_1'].map(community_map)
dfg['community_node_2'] = dfg['node_2'].map(community_map)


In [88]:
community_count_node_1 = dfg['community_node_1'].value_counts(dropna=False)

# Count the number of nodes in each community for 'community_node_2'
community_count_node_2 = dfg['community_node_2'].value_counts(dropna=False)

# Output the results
# print("Community counts for 'community_node_1':")
# print(community_count_node_1)
# print("\nCommunity counts for 'community_node_2':")
# print(community_count_node_2)

Community counts for 'community_node_1':
community_node_1
1     553
7      90
4      18
5      10
10      9
8       8
3       6
11      6
2       4
6       4
9       1
Name: count, dtype: int64

Community counts for 'community_node_2':
community_node_2
1     553
7      90
4      18
5      10
10      9
8       8
3       6
11      6
2       4
6       4
9       1
Name: count, dtype: int64


In [89]:
dfg.to_csv('updated_graph.csv', index=False)

# Creating an Agent

## Reads one Line of the CSV and generates test script

In [90]:
final_stretch = pd.read_csv('updated_graph.csv')    

In [91]:
final_stretch.columns

Index(['node_1', 'node_2', 'chunk_id', 'edge', 'count', 'path',
       'community_node_1', 'community_node_2'],
      dtype='object')

In [120]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.agents.initialize import initialize_agent
from langchain.tools import Tool
from langchain_core.prompts import PromptTemplate
import pandas as pd
import os
import ast
import logging
# Load the CSV file
df = pd.read_csv('updated_graph.csv')

# Custom tool to analyze CSV data
def analyze_csv(query):
    if "first row" in query:
        return df.iloc[0].to_dict()
    elif "column names" in query:
        return list(df.columns)
    elif "row count" in query:
        return len(df)
    else:
        return "Query not recognized. Please ask about 'first row', 'column names', or 'row count'."

csv_analyzer_tool = Tool(
    name="CSVAnalyzer",
    func=analyze_csv,
    description="Analyzes the CSV data. You can ask about 'first row', 'column names', or 'row count'."
)

# Custom tool to read file contents
def read_file_contents(file_path):
    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            return file.read()
    return f"File not found: {file_path}"

file_reader_tool = Tool(
    name="FileReader",
    func=read_file_contents,
    description="Reads the contents of a file given its path"
)

# Custom tool to generate test automation scripts
def generate_test_script(inputs):
    print("#############################################")
    try:
        dictionary = ast.literal_eval(inputs)
    except:
        dictionary['node_1'] = 'node_1'
        dictionary['node_2'] = 'node_2'
        dictionary['edge'] = 'edge'
        dictionary['path'] = 'path'
    print(type(dictionary))
    print("#############################################")
    node_1 = dictionary['node_1']
    node_2 = dictionary['node_2']
    edge = dictionary['edge']
    path = dictionary['path']
    
    print(f"generate_test_script called with node_1: {node_1}, node_2: {node_2}, edge: {edge}, path: {path}")
    
    template = """
    Create a Python test automation script for the following scenario:
    - Node 1: {node_1}
    - Node 2: {node_2}
    - Edge: {edge}
    - Path: {path}

    The script should:
    1. Set up the test environment
    2. Read the file contents from the given path
    3. Simulate the data flow between Node 1 and Node 2
    4. Verify the edge condition
    5. Clean up the test environment

    Please provide the complete Python script.
    """
    prompt = PromptTemplate(
        input_variables=["node_1", "node_2", "edge", "path"],
        template=template
    )
    return llm.invoke(prompt.format(node_1=node_1, node_2=node_2, edge=edge, path=path))

test_generator_tool = Tool(
    name="TestGenerator",
    func=generate_test_script,
    description="Generates a test automation script given inputs in dictionary form with the keys 'node_1', 'node_2', 'edge', and 'path'"
)

# Create the main agent
tools = [csv_analyzer_tool, file_reader_tool, test_generator_tool]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Example usage
result = agent.run("Analyze the CSV data to get the first row, then generate a test automation script based on that information.")
print(result)




> Entering new AgentExecutor chain...
 I need to first analyze the CSV data and get the first row, then generate a test automation script using the first row as input.
Action: CSVAnalyzer
Action Input: first row
Observation: {'node_1': '((libspdm_context_t*)spdm_context)->local_context.local_cert_chain_provision_size[0]', 'node_2': 'libspdm_read_responder_public_certificate_chain', 'chunk_id': '41c0a30e7e5a4bd1ae7bd8cc63a2d818,41c0a30e7e5a4bd1ae7bd8cc63a2d818,41c0a30e7e5a4bd1ae7bd8cc63a2d818,41c0a30e7e5a4bd1ae7bd8cc63a2d818,41c0a30e7e5a4bd1ae7bd8cc63a2d818', 'edge': 'contextual proximity', 'count': 5, 'path': 'vram\\SecurityPkg\\DeviceSecurity\\SpdmLib\\libspdm\\unit_test\\test_spdm_requester\\challenge.c', 'community_node_1': 1, 'community_node_2': 1}
Thought: Now I have the first row of the CSV data, I can use this information to generate a test automation script.
Action: TestGenerator
Action Input: {'node_1': '((libspdm_context_t*)spdm_context)->local_context.local_cert_chain_prov

## Reads a Section of the CSV and generates test script

In [12]:
from langchain.agents.agent_types import AgentType
from langchain.agents.initialize import initialize_agent
from langchain.tools import Tool
from langchain_core.prompts import PromptTemplate
import pandas as pd
import os
import ast
import re
import time
import logging

logging.basicConfig(level=logging.INFO)
# Load the CSV file
try:
    df = pd.read_csv('updated_graph.csv')
except Exception as e:
    print(f"Error loading CSV file: {e}")
    df = pd.DataFrame()  # Create an empty DataFrame if file loading fails

# Function to get a section of the CSV
def get_csv_section(df, start_row, end_row):
    try:
        df_filtered = df.drop(columns=['chunk_id'])
        return df_filtered.iloc[start_row:end_row].to_dict(orient='records')
    except Exception as e:
        print(f"Error getting CSV section: {e}")
        return []

# Custom tool to read file contents
def read_file_contents(file_path):
    try:
        # Remove any leading or trailing quotes from the file_path
        file_path = file_path.strip("'\"")
        
        # Construct the full path
        full_path = os.path.join(os.getcwd(), 'TestRepositories', file_path)
        
        # Normalize the path to handle any .. or . in the path
        full_path = os.path.normpath(full_path)
        
        if os.path.exists(full_path):
            with open(full_path, 'r', encoding='utf-8', errors='ignore') as file:
                logging.info(f"Reading file: {full_path}")
                return file.read()[:10000]  # Truncate to 10000 characters
        return f"File not found: {full_path}"
    except Exception as e:
        return f"Error reading file: {e}"

file_reader_tool = Tool(
    name="FileReader",
    func=read_file_contents,
    description="Reads the contents of a file given its path"
)
def write_markdown_section(identified_nodes, test_scripts, output_file="output.md", append=False):
    mode = "a" if append else "w"
    with open(output_file, mode) as f:
        if not append:
            f.write("# LangChain Test Automation Results\n\n")
        
        f.write("## Identified Nodes\n\n")
        for node in identified_nodes:
            f.write(f"- Node 1: {node['node_1']}\n")
            f.write(f"  Node 2: {node['node_2']}\n")
            f.write(f"  Edge: {node['edge']}\n")
            f.write(f"  Priority: {node['priority']}\n\n")
        
        f.write("## Generated Test Scripts\n\n")
        for i, script in enumerate(test_scripts, 1):
            f.write(f"### Script {i}\n\n")
            f.write("```python\n")
            f.write(script)
            f.write("\n```\n\n")
            f.write("---\n\n")
# Custom tool to generate test automation scripts
def generate_test_script(inputs):
    try:
        dictionary = ast.literal_eval(inputs)
        node_1 = dictionary['node_1']
        node_2 = dictionary['node_2']
        edge = dictionary['edge']
        path = dictionary['path']
        file_contents = dictionary.get('file_contents', 'File contents not provided')[:10000]  # Truncate to 10000 characters

        template = """<\s>[INST]
        Create a Python test automation script for the following scenario:
        - Node 1: {node_1}
        - Node 2: {node_2}
        - Edge: {edge}
        - Path: {path}
        - File Contents: {file_contents}

        Note: contextual proximity implies that while creating the graph they appeared in the same chunk of text.
        The script should:
        1. Set up the test environment.
        2. Read the file contents from the given path if you need more information.
        3. Provide detailed test cases for testing the file and its impact on a system level.
        4. Write comperehensive tests for the edge with respect to the file contents or assume data if necessary.
        5. Clean up the test environment.

        Please provide the complete Python script(assume data if necessary).[/INST]
        """
        prompt = PromptTemplate(
            input_variables=["node_1", "node_2", "edge", "path", 'file_contents'],
            template=template
        )
        return llm.invoke(prompt.format(node_1=node_1, node_2=node_2, edge=edge, path=path, file_contents=file_contents))[:10000]  # Truncate to 10000 characters
    except Exception as e:
        return f"Error generating test script: {e}"

test_generator_tool = Tool(
    name="TestGenerator",
    func=generate_test_script,
    description="Generates a test automation script given inputs in dictionary form with the keys 'node_1', 'node_2', 'edge', and 'path'"
)

# Create the main agent
tools = [file_reader_tool, test_generator_tool]

try:
    agent = initialize_agent(
        tools,
        llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=False,
        handle_parsing_errors=True
    )
except Exception as e:
    print(f"Error initializing agent: {e}")
    agent = None

# Function to extract list of dictionaries from LLM output
def extract_dicts_from_llm_output(output):
    try:
        # Find all dictionary-like structures in the output
        dict_pattern = r'\{[^{}]*\}'
        dict_strings = re.findall(dict_pattern, output)
        
        # Parse each dictionary-like string
        dicts = []
        for d_str in dict_strings:
            try:
                d = ast.literal_eval(d_str)
                if isinstance(d, dict) and all(key in d for key in ['node_1', 'node_2', 'edge', 'priority', 'path']):
                    dicts.append(d)
            except:
                pass  # Ignore any strings that can't be parsed as dictionaries
        
        return dicts
    except Exception as e:
        print(f"Error extracting dictionaries from LLM output: {e}")
        return []

# Process the CSV in sections
section_size = 10
total_rows = len(df)
test_scripts = []
identified_nodes = []

# Time tracking variables
start_time = time.time()
total_sections = (total_rows + section_size - 1) // section_size

for section, start_row in enumerate(range(0, total_rows, section_size), 1):
    section_start_time = time.time()
    try:
        end_row = min(start_row + section_size, total_rows)
        csv_section = get_csv_section(df, start_row, end_row)

        # Calculate estimated time and elapsed time
        elapsed_time = time.time() - start_time
        estimated_total_time = (elapsed_time / section) * total_sections
        estimated_time_left = estimated_total_time - elapsed_time

        print(f"\nProcessing rows {start_row} to {end_row - 1} (Section {section}/{total_sections}):")
        print(f"Elapsed time: {elapsed_time:.2f} seconds")
        print(f"Estimated time left: {estimated_time_left:.2f} seconds")

        # Ask the LLM to identify relevant sections and generate test scripts
        llm_context = f"""<\s>[INST]
        Here is a section of the CSV file (rows {start_row} to {end_row - 1}):

        {csv_section}

        Select which values of nodes, edges, paths are most relevant for SYSTEM LEVEL test automation scripts.Do not repeat nodes and edges.
        Assign them high or low priority for testing.
        Write them in dictionary format with keys: 'node_1','node_2','edge','priority','path'.
        Provide your selections as a list of dictionaries.[/INST]
        """[:10000]  # Truncate to 10000 characters

        llm_plan = llm.invoke(llm_context)
        # print(f"LLM Plan for rows {start_row} to {end_row - 1}:")
        # print(llm_plan[:1000])  # Print only first 1000 characters of the plan

        # Extract the list of dictionaries from the LLM output
        plan_list = extract_dicts_from_llm_output(llm_plan)

        # Generate test scripts based on the plan
        for item in plan_list:
            node_1 = item['node_1']
            node_2 = item['node_2']
            edge = item['edge']
            path = item['path']
            priority = item['priority']
            
            identified_nodes.append({
                'node_1': node_1,
                'node_2': node_2,
                'edge': edge,
                'priority': priority
            })
            
            # Generate the test script
            script_input = str({
                'node_1': node_1,
                'node_2': node_2,
                'edge': edge,
                'path': path
            })[:10000]  # Truncate to 10000 characters
            if agent:
                result = agent.run(f"Use the TestGenerator tool to create a test script for: {script_input}")
                test_scripts.append(result[:10000])  # Truncate to 10000 characters
            else:
                print("Agent not initialized. Skipping test script generation.")

        write_markdown_section(identified_nodes, test_scripts, append=(section > 1))

        # Clear the lists for the next section
        identified_nodes = []
        test_scripts = []

        # Print time taken for this section
        section_time = time.time() - section_start_time
        print(f"Time taken for this section: {section_time:.2f} seconds")

    except Exception as e:
        print(f"Error processing rows {start_row} to {end_row - 1}: {e}")

# Write output to markdown file

total_time = time.time() - start_time
print(f"\nTotal processing time: {total_time:.2f} seconds")
print("Results have been written to output.md")


Processing rows 0 to 9 (Section 1/71):
Elapsed time: 0.01 seconds
Estimated time left: 0.46 seconds


INFO:httpx:HTTP Request: POST https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1/completions "HTTP/1.1 200 OK"
INFO:root:Reading file: c:\Users\Hetansh_Shah\OneDrive - Dell Technologies\Documents\Dependancy_Finder\TestRepositories\vram\SecurityPkg\DeviceSecurity\SpdmLib\libspdm\unit_test\test_spdm_requester\challenge.c
INFO:httpx:HTTP Request: POST https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://opensour

Time taken for this section: 177.42 seconds

Processing rows 10 to 19 (Section 2/71):
Elapsed time: 177.42 seconds
Estimated time left: 6121.00 seconds


INFO:httpx:HTTP Request: POST https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1/completions "HTTP/1.1 200 OK"
INFO:htt

Time taken for this section: 180.26 seconds

Processing rows 20 to 29 (Section 3/71):
Elapsed time: 357.68 seconds
Estimated time left: 8107.43 seconds


...

Time taken for this section: 357.54 seconds

Processing rows 650 to 659 (Section 66/71):
Elapsed time: 24381.61 seconds
Estimated time left: 1847.09 seconds


INFO:httpx:HTTP Request: POST https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1/completions "HTTP/1.1 200 OK"
INFO:htt

Time taken for this section: 665.35 seconds

Processing rows 660 to 669 (Section 67/71):
Elapsed time: 25046.97 seconds
Estimated time left: 1495.34 seconds


INFO:httpx:HTTP Request: POST https://opensource-challenger-api.prdlvgpu1.aiaccel.dell.com/v1/completions "HTTP/1.1 200 OK"


KeyboardInterrupt: 